# HR Analytics Automation
This notebook demonstrates how to generate an automated HR report from an Excel dataset using Python.

In [1]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font
from openpyxl.utils.dataframe import dataframe_to_rows

# Load the input Excel data
df = pd.read_excel("hr_employee_data.xlsx")
df.head()


,Employee ID,Department,Hire Date,Salary,Status,Exit Date
0,E1000,Marketing,2022-05-22,31802,Exited,2026-02-23
1,E1001,Sales,2020-09-08,38155,Active,NaT
2,E1002,Finance,2016-10-15,38120,Active,NaT
3,E1003,Sales,2015-04-20,69384,Active,NaT
4,E1004,Sales,2016-05-22,77025,Active,NaT


## Step 1: Calculate Key Metrics

In [2]:
report = {}
report['Total Employees'] = len(df)
report['Active Employees'] = (df['Status'] == 'Active').sum()
report['Exited Employees'] = (df['Status'] == 'Exited').sum()
report['Average Salary'] = df['Salary'].mean()
report


{'Total Employees': 100,
 'Active Employees': 67,
 'Exited Employees': 33,
 'Average Salary': 59197.89}

## Step 2: Compute Tenure for Exited Employees

In [3]:
df['Tenure (years)'] = ((df['Exit Date'] - df['Hire Date']).dt.days / 365).round(1)
avg_tenure_by_dept = df[df['Status'] == 'Exited'].groupby('Department')['Tenure (years)'].mean().reset_index()
avg_tenure_by_dept


,Department,Tenure (years)
0,Finance,2.728571
1,HR,3.016667
2,IT,3.400000
3,Marketing,3.166667
4,Sales,2.800000


## Step 3: Calculate Turnover Rate by Department

In [4]:
turnover_by_dept = df.groupby('Department')['Status'].value_counts().unstack().fillna(0)
turnover_by_dept['Turnover Rate (%)'] = (turnover_by_dept['Exited'] / (turnover_by_dept['Active'] + turnover_by_dept['Exited']) * 100).round(1)
turnover_by_dept = turnover_by_dept.reset_index()[['Department', 'Turnover Rate (%)']]
turnover_by_dept


Status,Department,Turnover Rate (%)
0,Finance,43.8
1,HR,28.6
2,IT,27.8
3,Marketing,34.6
4,Sales,31.6


## Step 4: Export Final Report to Excel

In [5]:
wb = Workbook()
ws1 = wb.active
ws1.title = "HR Summary"
ws1["A1"] = "HR Summary Report"
ws1["A1"].font = Font(bold=True, size=14)

row = 3
for key, value in report.items():
    ws1[f"A{row}"] = key
    ws1[f"B{row}"] = round(value, 2) if isinstance(value, float) else value
    row += 1

ws2 = wb.create_sheet("Tenure by Department")
for r in dataframe_to_rows(avg_tenure_by_dept, index=False, header=True):
    ws2.append(r)

ws3 = wb.create_sheet("Turnover Rate")
for r in dataframe_to_rows(turnover_by_dept, index=False, header=True):
    ws3.append(r)

wb.save("automated_hr_report.xlsx")
print("Saved automated_hr_report.xlsx")


Saved automated_hr_report.xlsx
